In [1]:
from glob import glob
import numpy as np
import rasterio as rio
import os
import pandas as pd

# Importing dataset

In [2]:
root = '.\\steel defect'

In [3]:
images = glob(os.path.join(root, "images", '*.jpg'))

In [4]:
dataset = pd.read_csv('steel defect\\train.csv')


In [5]:
X = []
y = []
for index in range(dataset.shape[0]):
    for i in images:
        image_name = os.path.basename(i)
        if image_name!=dataset['ImageId'].iloc[index]:
            continue
        with rio.open(i, 'r') as f:
            X.append(f.read(1))
            y.append(int(dataset['ClassId'].iloc[index]))
        break  

C:\Users\Mihir\anaconda3\lib\site-packages\rasterio\__init__.py:228: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


# stack data

In [6]:
X = np.stack(X)
y = np.array(y)

# Flatten image array

In [7]:
n_samples = X.shape[0]
X_data = X.reshape((n_samples, -1))

# Data normalization

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_data = sc.fit_transform(X_data)

# Split the dataset into train and test

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data,y, test_size = 0.2, random_state = 1)

# <span style="color: red;">Random Forest Classifier</span>

In [10]:
from sklearn.ensemble import RandomForestClassifier
clf_rndm= RandomForestClassifier(n_estimators = 1000)
clf_rndm.fit(X_train, y_train)
rndm_pred = clf_rndm.predict(X_test)

In [11]:
from sklearn import metrics 


print("Confusion matrix : \n",metrics.confusion_matrix(rndm_pred,y_test))
print("Accuracy Score : ",metrics.accuracy_score(rndm_pred,y_test))
print("Classification report : \n",metrics.classification_report(rndm_pred,y_test))

Confusion matrix : 
 [[ 20   2   6   0]
 [  6  10   1   0]
 [ 66   3 422  64]
 [  0   0  17   3]]
Accuracy Score :  0.7338709677419355
Classification report : 
               precision    recall  f1-score   support

           1       0.22      0.71      0.33        28
           2       0.67      0.59      0.62        17
           3       0.95      0.76      0.84       555
           4       0.04      0.15      0.07        20

    accuracy                           0.73       620
   macro avg       0.47      0.55      0.47       620
weighted avg       0.88      0.73      0.79       620



# <span style="color:red;">SVM classifier</span>

In [13]:
from sklearn.svm import SVC
svm_classifier = SVC(kernel='rbf', random_state=0)#radial basis function
svm_classifier.fit(X_train,y_train)
svm_predict = svm_classifier.predict(X_test)

In [14]:
from sklearn import metrics
print("Confusion matrix : \n",metrics.confusion_matrix(svm_predict,y_test))
print("Accuracy Score : ",metrics.accuracy_score(svm_predict,y_test))
print("Classification report : \n",metrics.classification_report(svm_predict,y_test))

Confusion matrix : 
 [[ 20   0   2   0]
 [  1   5   1   0]
 [ 71  10 443  65]
 [  0   0   0   2]]
Accuracy Score :  0.7580645161290323
Classification report : 
               precision    recall  f1-score   support

           1       0.22      0.91      0.35        22
           2       0.33      0.71      0.45         7
           3       0.99      0.75      0.86       589
           4       0.03      1.00      0.06         2

    accuracy                           0.76       620
   macro avg       0.39      0.84      0.43       620
weighted avg       0.96      0.76      0.83       620



# <span style="color:red;">KNN classifier</span>

In [15]:
neighbors = list(range(2,9,1))

In [16]:
from sklearn.neighbors import KNeighborsClassifier

for k in neighbors:
    print("For k = ",k)
    knn_classifier = KNeighborsClassifier(n_neighbors = k, p=2)
    knn_classifier.fit(X_train, y_train)
    knn_pred = knn_classifier.predict(X_test)
    print("Confusion matrix : \n",metrics.confusion_matrix(knn_pred,y_test))
    print("Accuracy Score : ",metrics.accuracy_score(knn_pred,y_test))
    print("Classification report : \n",metrics.classification_report(knn_pred,y_test))
    print("\n")

For k =  2
Confusion matrix : 
 [[ 72   6  69   5]
 [  7   8  10   0]
 [ 13   1 362  61]
 [  0   0   5   1]]
Accuracy Score :  0.714516129032258
Classification report : 
               precision    recall  f1-score   support

           1       0.78      0.47      0.59       152
           2       0.53      0.32      0.40        25
           3       0.81      0.83      0.82       437
           4       0.01      0.17      0.03         6

    accuracy                           0.71       620
   macro avg       0.54      0.45      0.46       620
weighted avg       0.79      0.71      0.74       620



For k =  3
Confusion matrix : 
 [[ 59   4  33   3]
 [  8   9   8   0]
 [ 25   2 397  58]
 [  0   0   8   6]]
Accuracy Score :  0.7596774193548387
Classification report : 
               precision    recall  f1-score   support

           1       0.64      0.60      0.62        99
           2       0.60      0.36      0.45        25
           3       0.89      0.82      0.86       482
   